In [1]:
from scipy.io import loadmat
import matplotlib.pyplot as plt
import h5py
import numpy as np

%matplotlib inline

data['trial'][0] to access data
data['trial'][0].shape = 350  - these are the trials

In one trial:
data['trial'][0][x][0] - one number (60-68), session

data['trial'][0][x][1] - one number, stimAmp

data['trial'][0][x][2] - 'hit' or 'miss', response

data['trial'][0][x][3] - lick times

data['trial'][0][x][4] - (180,115) time*ca2+

In [2]:
def mat_to_h5py(filename, roi=False):
    print('Converting '+filename)
    data = loadmat(filename)
    f = h5py.File(filename+".hdf5", "w")
    if roi==False:
        n_trials = data['trial'][0].shape[0]
        experiment_number = data['trial'][0][0][0].shape[0]
        stim_strength = data['trial'][0][0][1].shape[0]
        hit_miss = data['trial'][0][0][2].shape[0]
        
        lick_times = 0                     #extract maximum number of lick times
        for i in range(n_trials):
            if data['trial'][0][i][3].shape[1] > lick_times:
                lick_times = data['trial'][0][i][3].shape[1]
        
        #print(lick_times)
        
        n_dendrites = data['trial'][0][0][4].shape[1]
        times = data['trial'][0][0][4].shape[0]
        
        print(n_dendrites, times)
        
        f.create_dataset("meta", (n_trials, experiment_number + stim_strength + hit_miss))
        if lick_times != 0:
            f.create_dataset("licktimes", (n_trials, lick_times)) # problem here
            f['licktimes'][:] = np.ones((n_trials, lick_times))*(-1)
        f.create_dataset("data", (n_trials, n_dendrites, times), dtype='float64')
        
        meta = np.zeros((n_trials, experiment_number + stim_strength + hit_miss))
        for i in range(n_trials):
            meta[i][0] = data['trial'][0][i][0][0][0] #experiment number
            meta[i][1] = data['trial'][0][i][1][0][0] #stim_strength
            if data['trial'][0][i][2][0]=='hit':
                meta[i][2] = 1    #hit/miss
            else:
                meta[i][2] = 0
        
        for i in range(n_trials):
            #f["meta"][i] = data['trial'][0][i][0][0][0]
            f['data'][i] = data['trial'][0][i][4].T
            #f['meta'][i][0] = data['trial'][0][i][0][0][0] #experiment number
            #f['meta'][i][1] = data['trial'][0][i][1][0][0] #stim_strength
            try:
                f['licktimes'][i,:data['trial'][0][i][3][0].shape[0]] = data['trial'][0][i][3][0]
            except IndexError:
                pass
        
        f['meta'][:] = meta

In [3]:
filelist = ["calcium_data/140708B_140811a_result", 
            "calcium_data/140909C_141112a_result", 
            "calcium_data/141006C_141121a_result",
            "calcium_data/150109A_150302a_result", 
            "calcium_data/151122A_160202a_result", 
            "calcium_data/151122B_160207a_result", 
            "calcium_data/160209A_160430a_result", 
            "calcium_data/160209B_160428a_result"]

In [4]:
for filename in filelist:
    mat_to_h5py(filename)

Converting calcium_data/140708B_140811a_result
115 180
Converting calcium_data/140909C_141112a_result
101 180
Converting calcium_data/141006C_141121a_result
99 180
Converting calcium_data/150109A_150302a_result
84 180
Converting calcium_data/151122A_160202a_result
123 180
Converting calcium_data/151122B_160207a_result
135 180
Converting calcium_data/160209A_160430a_result
86 180
Converting calcium_data/160209B_160428a_result
91 180


In [5]:
f = h5py.File(filelist[2]+".hdf5", "r")
f['data'][:].shape

(210, 99, 180)

In [27]:
f['meta'][3]

array([ 39.        ,   1.33333337,   1.        ], dtype=float32)

In [30]:
data = loadmat(filelist[2])
data['trial'][0][3][2]

array(['hit'], 
      dtype='<U3')